In [20]:
#imports
import math
import io 

In [109]:
#data pre-processing
data = []
with io.open('Sensorless_drive_diagnosis.txt', encoding='latin-1') as myfile:
    for i in myfile.readlines():
        data.append(i)
dataa= []
for i in data:
    i = i.split(" ")
    for j in range(len(i)):
        i[j] = float(i[j])
    dataa.append(i)
data = dataa
len(data)
test = data[0:7000]

In [22]:
#split 
def split(data,column,value):
    l,r = [],[]
    for row in data:
        if row[column] >= value:
            l.append(row)
        else:
            r.append(row)
    return l,r

#entropy of split
def entropy_split(groups,classes):
    ent = 0
    for group in groups:
        normalized_group_size = len(group)/len(data)
        group_sum = 0
        for i in classes:
            if len(group)==0:
                continue
            p = [row[-1] for row in group].count(i)/len(group)
            if p == 0:
                continue
            group_sum -= p*math.log2(p)
        ent += normalized_group_size * group_sum
    return ent

#best_split
def best_split(rows):
    entropy_dataset = 0
    class_values = list(set(row[-1] for row in data))
    for i in class_values:
        p = [row[-1] for row in data].count(i) / len(data)
        if p == 0:
            continue
        entropy_dataset -= p*math.log2(p)
    data_entropy,best_info_gain,chosen_column,chosen_value = entropy_dataset,0,0,0
    for column in range(0,len(rows[0])-1):
        values = set([row[column] for row in rows])
        for value in values:
            class_values = list(set(row[-1] for row in data))
            left,right = split(rows,column,value)
            print(len(left))
            if len(left)==0 or len(right)==0:
                continue
            entropy_of_split = entropy_split([left,right],class_values)
            curr_info_gain = data_entropy - entropy_of_split
            if curr_info_gain > best_info_gain:
                best_info_gain = curr_info_gain
                chosen_column = column
                chosen_value = value
            print(best_info_gain)
    return chosen_column,chosen_value,best_info_gain

In [92]:
from random import randrange,sample
class Leaf:
    def __init__(self,rows):
        classes = list(set(row[-1] for row in rows))
        if len(classes)>1:
            ps =[]
            for i in range(0,len(classes)):
                p = [row[-1] for row in rows].count(classes[i]) / len(rows)
                ps.append(p)
            class_selected = max(ps).index()
            self.prediction = classes[class_selected]
        else:
            self.prediction = classes[0]
        
class Branch:
    def __init__(self,left,right,ix,val):
        self.ix = ix
        self.val = val
        self.left = left
        self.right = right
    def match(self,data):
        return data[self.ix]>self.val
class_values = list(set(row[-1] for row in data))

def shuffle(x):
    for i in range(0,x):
        rand1 = randrange(1000,25000)
        rand2 = randrange(25000,55000)
        temp = data[rand1] 
        data[rand1] = data[rand2]
        data[rand2] = temp

def build_tree(rows,ctr,max_depth=4):
    col,val,bif = best_split(rows)
    if ctr>max_depth:
        return Leaf(rows)
    elif bif == 0:
        return Leaf(rows)
    print(bif,len(rows),ctr)
    true,false = split(rows,col,val)
    ctr = ctr + 1
    left_twig = build_tree(true,ctr,max_depth)
    right_twig = build_tree(false,ctr,max_depth)
    return Branch(left_twig,right_twig,col,val)

In [93]:
def print_tree(node, spacing=""):
    if isinstance(node, Leaf):
        print (spacing + "Predict", node.prediction)
        return
    print (spacing + 'Is Column %s value >= %s'%(node.ix,node.val))
    # Call this function recursively on the true branch
    print (spacing + '--> True:')
    print_tree(node.left, spacing + "  ")

    # Call this function recursively on the false branch
    print (spacing + '--> False:')
    print_tree(node.right, spacing + "  ")

In [95]:
shuffle(10000)
train =[]
ixs = sample(range(17000, 47000), 100)
for i in ixs:
    train.append(data[i])
dt = build_tree(train,0,7)
print_tree(dt)

46
3.4540096348339553
68
3.454014269486997
70
3.454014269486997
10
3.454014269486997
7
3.454014269486997
48
3.454014269486997
3
3.454014269486997
4
3.454014269486997
72
3.4540727421917365
43
3.4540727421917365
52
3.4540727421917365
77
3.4540727421917365
35
3.4540727421917365
76
3.4540727421917365
22
3.4540727421917365
90
3.4540727421917365
65
3.4540727421917365
54
3.4540727421917365
18
3.4540727421917365
1
3.4540727421917365
58
3.4540727421917365
100
71
3.4540727421917365
64
3.4540727421917365
16
3.4540727421917365
81
3.4540727421917365
97
3.4540727421917365
20
3.4540727421917365
8
3.4540727421917365
69
3.4540727421917365
59
3.4540727421917365
66
3.4540727421917365
34
3.4540727421917365
56
3.4540727421917365
27
3.4540727421917365
5
3.4540727421917365
57
3.4540727421917365
12
3.4540727421917365
62
3.4540727421917365
95
3.4540727421917365
89
3.4540727421917365
79
3.4540727421917365
21
3.4540727421917365
28
3.4540727421917365
26
3.4540727421917365
36
3.4540727421917365
61
3.45407274219173

62
3.454356916141525
76
3.454356916141525
87
3.454356916141525
24
3.454356916141525
2
3.454356916141525
6
3.454356916141525
25
3.454356916141525
19
3.454356916141525
84
3.454356916141525
49
3.454356916141525
91
3.454356916141525
51
3.454356916141525
2
3.454356916141525
16
3.454356916141525
100
24
3.454356916141525
53
3.454356916141525
35
3.454356916141525
45
3.454356916141525
33
3.454356916141525
42
3.454356916141525
13
3.454356916141525
56
3.454356916141525
50
3.454356916141525
3
3.454356916141525
65
3.454356916141525
40
3.454356916141525
31
3.454356916141525
75
3.454356916141525
59
3.454356916141525
32
3.454356916141525
48
3.454356916141525
52
3.454356916141525
83
3.454356916141525
80
3.454356916141525
26
3.454356916141525
90
3.454356916141525
62
3.454356916141525
29
3.454356916141525
28
3.454356916141525
5
3.454356916141525
18
3.454356916141525
8
3.454356916141525
9
3.454356916141525
58
3.454356916141525
41
3.454356916141525
87
3.454356916141525
55
3.454356916141525
37
3.45435691614

37
3.455030965339586
40
3.455030965339586
27
3.455030965339586
88
3.455030965339586
90
3.455030965339586
83
3.455030965339586
46
3.455030965339586
66
3.455030965339586
31
3.455030965339586
97
3.455030965339586
70
3.455030965339586
54
3.455030965339586
55
3.455030965339586
53
3.455030965339586
82
3.455030965339586
30
3.455030965339586
35
3.455030965339586
63
3.455030965339586
41
3.455030965339586
36
3.455030965339586
62
3.455030965339586
93
3.455030965339586
39
3.455030965339586
32
3.455030965339586
43
3.455030965339586
7
3.455030965339586
34
3.455030965339586
95
3.455030965339586
18
3.455030965339586
21
3.455030965339586
88
3.455030965339586
64
3.455030965339586
29
3.455030965339586
10
3.455030965339586
44
3.455030965339586
5
3.455030965339586
68
3.455030965339586
12
3.455030965339586
61
3.455030965339586
81
3.455030965339586
20
3.455030965339586
92
3.455030965339586
65
3.455030965339586
69
3.455030965339586
28
3.455030965339586
72
3.455030965339586
76
3.455030965339586
78
3.4550309653

20
3.4555075009372174
38
3.4555075009372174
44
3.4555075009372174
57
3.4555075009372174
71
3.4555075009372174
77
3.4555075009372174
81
3.4555075009372174
2
3.4555075009372174
26
3.4555075009372174
12
3.4555075009372174
34
3.4555075009372174
14
3.4555075009372174
82
3.4555075009372174
62
3.4555075009372174
78
3.4555075009372174
24
3.4555075009372174
85
3.4555075009372174
37
3.4555075009372174
35
3.4555075009372174
25
3.4555075009372174
72
3.4555075009372174
42
3.4555075009372174
8
3.4555075009372174
64
3.4555075009372174
69
3.4555075009372174
23
3.4555075009372174
87
3.4555075009372174
81
3.4555075009372174
17
3.4555075009372174
13
3.4555075009372174
92
3.4555075009372174
90
3.4555075009372174
38
3.4555075009372174
51
3.4555075009372174
41
3.4555075009372174
45
3.4555075009372174
12
3.4555075009372174
9
3.4555075009372174
62
3.4555075009372174
36
3.4555075009372174
1
3.4555075009372174
59
3.4555075009372174
63
3.4555075009372174
39
3.4555075009372174
55
3.4555075009372174
68
3.455507500

44
3.4555075009372174
26
3.4555075009372174
20
3.4555075009372174
98
3.4555075009372174
36
3.4555075009372174
95
3.4555075009372174
99
3.4555075009372174
62
3.4555075009372174
31
3.4555075009372174
55
3.4555075009372174
91
3.4555075009372174
9
3.4555075009372174
66
3.4555075009372174
77
3.4555075009372174
7
3.4555075009372174
93
3.4555075009372174
94
3.4555075009372174
1
3.4555075009372174
50
3.4555075009372174
27
3.4555075009372174
10
3.4555075009372174
52
3.4555075009372174
15
3.4555075009372174
40
3.4555075009372174
42
3.4555075009372174
75
3.4555075009372174
82
3.4555075009372174
68
3.4555075009372174
21
3.4555075009372174
69
3.4555075009372174
81
3.4555075009372174
59
3.4555075009372174
65
3.4555075009372174
31
3.4555075009372174
44
3.4555075009372174
52
3.4555075009372174
83
3.4555075009372174
86
3.4555075009372174
19
3.4555075009372174
70
3.4555075009372174
71
3.4555075009372174
45
3.4555075009372174
60
3.4555075009372174
24
3.4555075009372174
15
3.4555075009372174
75
3.45550750

...

4
2
3.459431618637298
4
1
3.459431618637298
2
3.459431618637298
3
3.459431618637298
3
3.459431618637298
2
3.459431618637298
4
1
3.459431618637298
1
3.459431618637298
3
3.459431618637298
4
2
3.459431618637298
2
3.459431618637298
3
3.459431618637298
4
1
3.459431618637298
1
3.459431618637298
3
3.459431618637298
4
2
3.459431618637298
3
3.459431618637298
1
3.459431618637298
4
2
3.459431618637298
2
3.459431618637298
4
1
3.459431618637298
3
3.459431618637298
4
3
3.459431618637298
2
3.459431618637298
1
3.459431618637298
4
3
3.459431618637298
1
3.459431618637298
2
3.459431618637298
1
3.459431618637298
3
3.459431618637298
2
3.459431618637298
4
1
3.459431618637298
3
3.459431618637298
2
3.459431618637298
4
1
3.459431618637298
3
3.459431618637298
2
3.459431618637298
4
4
3
3.459431618637298
1
3.459431618637298
2
3.459431618637298
4
3
3.459431618637298
1
3.459431618637298
2
3.459431618637298
4
3
3.459431618637298
1
3.459431618637298
2
3.459431618637298
3.459431618637298 4 4
3
2
3.459431618637298
1
3.

9
3.4593974358619985
8
3.4593974358619985
7
3.4593974358619985
5
3.4593974358619985
1
3.4593974358619985
2
3.4593974358619985
10
4
3.4593974358619985
6
3.4593974358619985
9
3.4593974358619985
8
3.4593974358619985
3
3.4593974358619985
5
3.4593974358619985
10
4
3.4593974358619985
1
3.4593974358619985
6
3.4593974358619985
8
3.4593974358619985
7
3.4593974358619985
2
3.4593974358619985
3
3.4593974358619985
9
3.4593974358619985
5
3.4593974358619985
7
3.4593974358619985
8
3.4593974358619985
6
3.4593974358619985
3
3.4593974358619985
9
3.4593974358619985
10
4
3.4593974358619985
2
3.4593974358619985
1
3.4593974358619985
2
3.4593974358619985
1
3.4593974358619985
8
3.4593974358619985
9
3.4593974358619985
7
3.4593974358619985
6
3.4593974358619985
10
3
3.4593974358619985
4
3.4593974358619985
5
3.4593974358619985
5
3.4593974358619985
9
3.4593974358619985
7
3.4593974358619985
6
3.4593974358619985
2
3.4593974358619985
4
3.4593974358619985
1
3.4593974358619985
10
3
3.4593974358619985
8
3.459397435861998

8
5
3.459431618637298
2
3.459431618637298
6
3.459431618637298
1
3.459431618637298
6
3.459431618637298
5
3.459431618637298
1
3.459431618637298
3
3.459431618637298
4
3.459431618637298
2
3.459431618637298
8
7
3.459431618637298
5
3.459431618637298
1
3.459431618637298
2
3.459431618637298
6
3.459431618637298
4
3.459431618637298
8
7
3.459431618637298
3
3.459431618637298
5
3.459431618637298
8
4
3.459431618637298
1
3.459431618637298
6
3.459431618637298
7
3.459431618637298
2
3.459431618637298
3
3.459431618637298
5
3.459431618637298
7
3.459431618637298
6
3.459431618637298
3
3.459431618637298
8
4
3.459431618637298
2
3.459431618637298
1
3.459431618637298
2
3.459431618637298
1
3.459431618637298
7
3.459431618637298
8
6
3.459431618637298
3
3.459431618637298
4
3.459431618637298
5
3.459431618637298
5
3.459431618637298
6
3.459431618637298
2
3.459431618637298
4
3.459431618637298
1
3.459431618637298
8
3
3.459431618637298
7
3.459431618637298
3
3.459431618637298
5
3.459431618637298
7
3.459431618637298
2
3.45

4
3.459431618637298
1
3.459431618637298
2
3.459431618637298
5
2
3.459431618637298
4
3.459431618637298
3
3.459431618637298
1
3.459431618637298
5
4
3.459431618637298
3
3.459431618637298
2
3.459431618637298
1
3.459431618637298
2
3.459431618637298
1
3.459431618637298
3
3.459431618637298
5
4
3.459431618637298
5
2
3.459431618637298
1
3.459431618637298
3
3.459431618637298
4
3.459431618637298
2
3.459431618637298
1
3.459431618637298
5
3
3.459431618637298
4
3.459431618637298
1
3.459431618637298
2
3.459431618637298
3
3.459431618637298
4
3.459431618637298
5
5
2
3.459431618637298
1
3.459431618637298
3
3.459431618637298
4
3.459431618637298
2
3.459431618637298
1
3.459431618637298
5
3
3.459431618637298
4
3.459431618637298
3.459431618637298 5 5
2
1
3.459431618637298
1
3.459431618637298
2
1
3.459431618637298
2
2
1
3.459431618637298
1
3.459431618637298
2
1
3.459431618637298
2
1
3.459431618637298
2
1
3.459431618637298
2
1
3.459431618637298
2
2
1
3.459431618637298
2
1
3.459431618637298
2
1
3.45943161863729

2
1
3.459431618637298
2
1
3.459431618637298
2
1
3.459431618637298
2
1
3.459431618637298
2
1
3.459431618637298
1
3.459431618637298
2
1
3.459431618637298
2
1
3.459431618637298
2
3.459431618637298 2 6
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
3.459431618637298
1
3.459431618637298
2
1
3.459431618637298
2
2
1
3.459431618637298
1
3.459431618637298
2
1
3.459431618637298
2
1
3.459431618637298
2
1
3.459431618637298
2
1
3.459431618637298
2
1
3.459431618637298
2
1
3.459431618637298
2
1
3.459431618637298
2
2
1
3.459431618637298
2
1
3.459431618637298
1
3.459431618637298
2
1
3.459431618637298
2
2
1
3.459431618637298
2
1
3.459431618637298
2
1
3.459431618637298
2
1
3.459431618637298
2
1
3.459431618637298
2
1
3.459431618637298
2
1
3.459431618637298
2
1
3.45

In [96]:
def classify(row, node):
    if isinstance(node, Leaf):
        return node.prediction
    if node.match(row): 
        return classify(row, node.left)
    else:
        return classify(row, node.right)
classify(data[252],dt)

1.0

In [120]:
def accuracy(data,dt):
    x = 0
    for row in data:
           if(row[-1]==classify(row, dt)):
                x += 1
    print("Accuracy - ", (x/len(data))*100) 
test=data[10000:25000]
accuracy(test,dt)

Accuracy -  79.14


In [98]:
print_tree(dt)

Is Column 9 value >= -0.0036331
--> True:
  Is Column 9 value >= 0.033873
  --> True:
    Is Column 9 value >= 0.07212
    --> True:
      Is Column 0 value >= 1.0845e-06
      --> True:
        Is Column 0 value >= 1.0491e-05
        --> True:
          Predict 7.0
        --> False:
          Is Column 0 value >= 9.8112e-06
          --> True:
            Predict 7.0
          --> False:
            Predict 7.0
      --> False:
        Is Column 0 value >= -1.5728e-06
        --> True:
          Is Column 0 value >= -8.5195e-07
          --> True:
            Is Column 0 value >= 3.9337e-07
            --> True:
              Predict 7.0
            --> False:
              Predict 7.0
          --> False:
            Is Column 0 value >= -1.4431e-06
            --> True:
              Predict 7.0
            --> False:
              Predict 7.0
        --> False:
          Is Column 0 value >= -6.0544e-06
          --> True:
            Is Column 0 value >= -2.5824e-06
            -